In [137]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn import linear_model
import statistics

# FUNCTIONS

# 1) Delete

In [ ]:
#delete rows or colums with nan
# del_nan(df, col = False) - delete rows, if include nan, df - dataframe
def nan_detect(df):
    nan_pos = []
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            try:
                if  pd.isna(df.iloc[i,j]):             
                        nan_pos.append((i,j))
            except:
                pass
    return nan_pos

def del_nan(df, col = False):
    if col:
        df = df.drop(df.columns[[i[1] for i in nan_detect(df) ]], axis=1)

    else:
        df = df.drop(df.index[[i[0] for i in nan_detect(df) ]], axis = 0)

    return df

# 2) Replace nan with average, mode or mediana

In [ ]:
# replace nan with average, mode or mediana
# amm(a, n = 1, b = 'avarage'), n - number of column, a - dataframe
def amm(a, n = 1, b = 'avarage'):
    
    if b == 'avarage':
        
        s = 0    
        k = len(a)
    
        for i in range(len(a)): 
            if a[i][n] == a[i][n]:
                s = s + a[i][n]
            else:
                k -= 1        
        
        for i in range (len(a)):
            if a[i][n] != a[i][n]:
                a[i][n] = s/k
        return (a)
    
    elif b == 'mediana':
        
        l = []
        for i in range(len(a)):
            k = len(a)
            if a[i][n] == a[i][n]:            
                l.append(a[i][n])            
            else:
                k -= 1
        
        l.sort()
            
        for i in range (len(a)):
            if a[i][n] != a[i][n]:
                a[i][n] = l[len(l)//2]    
        return (a)
    
    elif b == 'moda':    
        
        l = []
        for i in range(len(a)):        
            k = len(a)        
            if a[i][n] == a[i][n]:            
                l.append(a[i][n])             
            else:
                k -= 1          
            
        for i in range (len(a)):
            if a[i][n] != a[i][n]:
                a[i][n] = max(l, key=l.count)    
        return (a)
    
    else:
        print ('please use avarage, moda or mediana')
        
        
        

# 3) Replace nan with linear regression result

In [204]:
# replace nan with linear regression result
#LR(a, n = 0), n - number of column

def LR(a, n = 0, k = 1):
    
    a1 = a
    
    #functions for del nan
    def nan_detect(df):
        nan_pos = []
        for i in range(df.shape[0]):
            for j in range(df.shape[1]):
                try:
                    if  pd.isna(df.iloc[i,j]):             
                        nan_pos.append((i,j))
                except:
                    pass
        return nan_pos
    
    def del_nan(df, col = False):
        if col:
            df = df.drop(df.columns[[i[1] for i in nan_detect(df) ]], axis=1)

        else:
            df = df.drop(df.index[[i[0] for i in nan_detect(df) ]], axis = 0)

        return df
    
    
    # Linear regression
    
    a = pd.DataFrame(a)
    a = del_nan(a)
    y = np.array(a[n])    
    X = np.array(a[k])
    
        
    #X = X.T 
    X = np.c_[X, np.ones(X.shape[0])] 
    beta_hat = np.linalg.lstsq(X,y)[0]
    
    beta_hat1 = beta_hat[-1]
    beta_hat = beta_hat[:-1]

  
    def listsum(numList):
        if len(numList) == 1:
            return numList[0]
        else:
            return numList[0] + listsum(numList[1:])  
        
    # new coeficients    
    
    for i in range (len(a1)):
        f = []
        
        if a1[i][n] != a1[i][n]:    
                              
            f.append(a1[i][k])                          
                     
            c = np.multiply(np.array(f),beta_hat1)   
                         
            a1[i][n] = (listsum(c) + beta_hat[-1])
            
    return a1
            


# KNN

In [146]:

def get_nan_list(column):
    from math import isnan
    nan_list = []
    for i in range(len(column)):
        if isnan(column[i]): 
            nan_list.append(i)
    return nan_list

def get_train_test(data, column_name, nan_list):
    all_list = list(range(0, data.shape[0]))
    not_nan_list = list(set(all_list)-set(nan_list))
    data_predict = data.iloc[nan_list, :]
    data_train = data.iloc[not_nan_list, :]
    return data_train, data_predict

def get_columns(data, column_name):
    columns = list(data.columns)
    del columns[columns.index(column_name)]
    return columns
    
def euclidean_distance(node1, node2, shape):
    import math
    distance = 0
    for x in shape:
        distance += pow((node1[x] - node2[x]), 2)
    return math.sqrt(distance)

def get_neighbors(data, column_name, amount_k):
    if data[column_name].dtype=='object':
        raise ValueError
    nan_list = get_nan_list(data[column_name])
    all_list = list(range(0, data.shape[0]))
    not_nan_list = list(set(all_list)-set(nan_list))
    if len(nan_list)==0:
        raise ValueError
    data_train, data_predict = get_train_test(data, column_name, nan_list)        
    columns = get_columns(data, column_name)
    all_neighbors = []
    for y in nan_list:
        distances = []
        for x in not_nan_list:
            dist = euclidean_distance(
                data_predict.loc[y, columns], 
                data_train.loc[x, columns], 
                columns)
            distances.append((x, dist))
        distances.sort(key=lambda x:x[1])
        all_neighbors.append(distances[:amount_k])  
        print(y)  
    return all_neighbors

def get_target(neighbors, target):
    sum_ = 0
    sum_d = 0
    for i in neighbors:
        if i[1]==0:
            return target[i[0]]
        sum_d +=1/i[1]
        sum_+=1/i[1]*target[i[0]]
    return sum_/sum_d



def replace_knn(data, column_name, amount_k):
    if data[column_name].dtype=='object':
        raise ValueError
    nan_list = get_nan_list(data[column_name])
    if len(nan_list)==0:
        raise ValueError
    predictions = []
    neighbors = get_neighbors(data, column_name, amount_k)
    for i in neighbors:
        predictions.append(get_target(i, data[column_name]))
    data_=data.copy(deep=True)
    for i in range(len(nan_list)):
        data_.loc[nan_list[i], column_name]=predictions[i]
    return data_


# Standardizes

In [140]:
#standardizes your data
#st(a, n = 0), a - data, n - number of column


def st(a, n = 0):
    
    l = []
    for i in range(len(a)):        
        if a[i][n] == a[i][n]:            
            l.append(a[i][n])            
    s = statistics.stdev(l) 
    m = statistics.mean(l) 
    
    
    for i in range (len(a)):
        if a[i][n] == a[i][n]:
            a[i][n] = (a[i][n] - m)/s
        else:
            print ("your data include nan")
    return (a)

# Scales

In [143]:
# scales values in your data
#msh(a, n = 0) - a your data, n - number of column


def msh(a, n = 0):
    
    l = []
    for i in range(len(a)):        
        if a[i][n] == a[i][n]:            
            l.append(a[i][n])            
       
        
    for i in range (len(a)):
        if a[i][n] == a[i][n]:
            a[i][n] = (a[i][n] - min(l))/(max(l)-min(l))
        else:
            print ("your data include nan")
    return (a)
       

# Data preprocessing

In [172]:
data = pd.read_csv('C:/Users/BIONIC Admin/Desktop/ML/winemag-data-130k-v2.csv', error_bad_lines=False)

### Data description
Датасет містить інформацію про вина і має наступні колонки:

country - країна, з якої вино

points - WineEnthusiast оцінювали вина за шкалою 1-100 

price - ціна за пляшку вина

province - провінція з якої вино

region_1 - Виробнича територія в провінції або штаті (напр. Napa)

region_2 - Іноді існують більш конкретні регіони, визначені в межах виноробної території (наприклад, Резерфорд в долині Напа), але це значення іноді може бути порожнім

title -  назва винної рецензії

variety - вид винограду, що використовується для виготовлення вина 

winery - ім'я виноробні, які зробили вино


### taster_twitter_handle - твітер винного знавця, який оцінював вино. Цю змінну ми і спробуємо прогнозувати. 
Фактично за інформацією про вино та поставленою оцінкою вгадати людину, яка оцінювала

In [173]:
data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [174]:
# describe dataframe 
data.describe()


,Unnamed: 0,points,price
count,129971.000000,129971.000000,120975.000000
mean,64985.000000,88.447138,35.363389
std,37519.540256,3.039730,41.022218
min,0.000000,80.000000,4.000000
25%,32492.500000,86.000000,17.000000
50%,64985.000000,88.000000,25.000000
75%,97477.500000,91.000000,42.000000
max,129970.000000,100.000000,3300.000000


In [175]:
# print dataframe info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
Unnamed: 0               129971 non-null int64
country                  129908 non-null object
description              129971 non-null object
designation              92506 non-null object
points                   129971 non-null int64
price                    120975 non-null float64
province                 129908 non-null object
region_1                 108724 non-null object
region_2                 50511 non-null object
taster_name              103727 non-null object
taster_twitter_handle    98758 non-null object
title                    129971 non-null object
variety                  129970 non-null object
winery                   129971 non-null object
dtypes: float64(1), int64(2), object(11)
memory usage: 8.4+ MB


### Deal with nans

In [176]:
# Check amount of nans
check_nans = pd.DataFrame({col:data[col].isna().sum() for col in data.columns}, index = ['Amount of NaNs'])
check_nans

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
Amount of NaNs,0,63,0,37465,0,8996,63,21247,79460,26244,31213,0,1,0


In [ ]:
# Delete rows, which colums include nans
data = del_nan(data, col = False)

### First model - only deleted nans 

In [129]:
#first 
# Make model only after delete nans 
data = pd.read_csv('C:/Users/BIONIC Admin/Desktop/ML/winemag-data-130k-v2.csv', error_bad_lines=False)

data = del_nan(data, col = False)

data  = data.iloc[1:10000]
targets = data['taster_twitter_handle'].astype('category')
data = data.drop(labels=['taster_twitter_handle'], axis=1)

true = ['points','price','region_1','winery','province','variety']
data_new = data[true]
data_new = pd.get_dummies(data_new)

X_train, X_test, y_train, y_test = train_test_split(data_new, targets, test_size=0.2, random_state=0)

lr = LogisticRegression()
lr = lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy_score(y_test, y_pred)

c:\users\bionic admin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bionic admin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8525

##### Accuracy score - 0.8525

### Second model - replace nans with avarage value

In [130]:
#avarage
data = pd.read_csv('C:/Users/BIONIC Admin/Desktop/ML/winemag-data-130k-v2.csv', error_bad_lines=False)

#convert data to list_of_lists - make avarage function - convert data to pd.Data Frame
data = data.values.tolist()
data = amm(data, n = 5, b = 'avarage')
data = pd.DataFrame(data)

#Drop nan in not numeric, Leave 100 000 rows
data = del_nan(data, col = False)
data  = data.iloc[1:10000]

# Get targets
targets = data[10].astype('category')

# Filter dataset, select few columns
data = data.drop(labels=[10], axis=1)
true = [4,5,7,13,6,12]
data_new = data[true]

# Get dummies in categorical
data_new = pd.get_dummies(data_new)

# Split data to train/test sets, Create classifier, Get accuracy score

X_train, X_test, y_train, y_test = train_test_split(data_new, targets, test_size=0.2, random_state=0)

lr = LogisticRegression()
lr = lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy_score(y_test, y_pred)

c:\users\bionic admin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bionic admin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.846

#### Accuracy score - 0.846

# Third model - replace nans with linear regression result

In [177]:
data = pd.read_csv('C:/Users/BIONIC Admin/Desktop/ML/winemag-data-130k-v2.csv', error_bad_lines=False)

#convert data to list_of_lists - make linear regression function - convert data to pd.Data Frame
data = data.values.tolist()
data = LR(data, n = 5, k = 4)
data = pd.DataFrame(data)

#Drop nan in not numeric, Leave 100 000 rows
data = del_nan(data, col = False)
data  = data.iloc[1:10000]

# Get targets
targets = data[10].astype('category')

# Filter dataset, select few columns
data = data.drop(labels=[10], axis=1)
true = [4,5,7,13,6,12]
data_new = data[true]

# Get dummies in categorical
data_new = pd.get_dummies(data_new)

# Split data to train/test sets, Create classifier, Get accuracy score

X_train, X_test, y_train, y_test = train_test_split(data_new, targets, test_size=0.2, random_state=0)

lr = LogisticRegression()
lr = lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy_score(y_test, y_pred)

c:\users\bionic admin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
c:\users\bionic admin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bionic admin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.847

#### Acurracy score 0.847

# Fourth model - standartazing + scaling

In [145]:
data = pd.read_csv('C:/Users/BIONIC Admin/Desktop/ML/winemag-data-130k-v2.csv', error_bad_lines=False)

#Drop nan in not numeric, Leave 100 000 rows
data = del_nan(data, col = False)
data  = data.iloc[1:10000]

#standartazing + scaling numeric data - points and price
data = data.values.tolist()
data = st(data, n = 5)
data = st(data, n = 4)

data = msh(data, n = 5)
data = msh(data, n = 4)

data = pd.DataFrame(data)


# Get targets
targets = data[10].astype('category')

# Filter dataset, select few columns
data = data.drop(labels=[10], axis=1)
true = [4,5,7,13,6,12]
data_new = data[true]

# Get dummies in categorical
data_new = pd.get_dummies(data_new)

# Split data to train/test sets, Create classifier, Get accuracy score

X_train, X_test, y_train, y_test = train_test_split(data_new, targets, test_size=0.2, random_state=0)

lr = LogisticRegression()
lr = lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy_score(y_test, y_pred)

c:\users\bionic admin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bionic admin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8565

#### Accuracy score - 0.8565

# KNN

In [209]:
# KNN

data = pd.read_csv('C:/Users/BIONIC Admin/Desktop/ML/winemag-data-130k-v2.csv', error_bad_lines=False)


one = ['points','price']
data = data[one]
data = replace_knn(data, 'price', 10)
#data = pd.DataFrame(data)

#Drop nan in not numeric, Leave 10 000 rows
data = del_nan(data, col = False)

data  = data.iloc[1:10000]
targets = data['taster_twitter_handle'].astype('category')
data = data.drop(labels=['taster_twitter_handle'], axis=1)

true = ['points','price','region_1','winery','province','variety']
data_new = data[true]
data_new = pd.get_dummies(data_new)

X_train, X_test, y_train, y_test = train_test_split(data_new, targets, test_size=0.2, random_state=0)

lr = LogisticRegression()
lr = lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy_score(y_test, y_pred)


c:\users\bionic admin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bionic admin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.846

#### Accuracy score 0.846

# Model - standartazing + scaling + linear regression

In [ ]:
data = pd.read_csv('C:/Users/BIONIC Admin/Desktop/ML/winemag-data-130k-v2.csv', error_bad_lines=False)



one = ['points','price']
data = data[one]
data = replace_knn(data, 'price', 10)
#data = pd.DataFrame(data)

#Drop nan in not numeric, Leave 10 000 rows
data = del_nan(data, col = False)

data  = data.iloc[1:10000]
targets = data['taster_twitter_handle'].astype('category')
data = data.drop(labels=['taster_twitter_handle'], axis=1)

true = ['points','price','region_1','winery','province','variety']
data_new = data[true]
data_new = pd.get_dummies(data_new)

X_train, X_test, y_train, y_test = train_test_split(data_new, targets, test_size=0.2, random_state=0)

lr = LogisticRegression()
lr = lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy_score(y_test, y_pred)


In [208]:
data = pd.read_csv('C:/Users/BIONIC Admin/Desktop/ML/winemag-data-130k-v2.csv', error_bad_lines=False)

#convert data to list_of_lists - make linear regression function - convert data to pd.Data Frame
data = data.values.tolist()
data = LR(data, n = 5, k = 4)
data = pd.DataFrame(data)

data = del_nan(data, col = False)

data = data.values.tolist()
data = st(data, n = 5)
data = st(data, n = 4)

data = msh(data, n = 5)
data = msh(data, n = 4)


data = pd.DataFrame(data)

#Drop nan in not numeric, Leave 100 000 rows

data  = data.iloc[1:10000]

# Get targets
targets = data[10].astype('category')

# Filter dataset, select few columns
data = data.drop(labels=[10], axis=1)
true = [4,5,7,13,6,12]
data_new = data[true]

# Get dummies in categorical
data_new = pd.get_dummies(data_new)

# Split data to train/test sets, Create classifier, Get accuracy score

X_train, X_test, y_train, y_test = train_test_split(data_new, targets, test_size=0.2, random_state=0)

lr = LogisticRegression()
lr = lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy_score(y_test, y_pred)

c:\users\bionic admin\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
c:\users\bionic admin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bionic admin\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.856

#### BEST accuracy - 0.8565 with dropping nan values, standartazing + scaling

##### But all accuracies are very close to each other